In [21]:
from utils.get_data import get_data

from recommandation.recommand_general import recommand_general
from recommandation.recommand_with_genre import recommand_with_genres
from recommandation.recommand_with_movie import recommand_with_movies
from recommandation.recommand_with_userid import recommand_user_id_ensemble

In [22]:
# get merged data
main_raw=get_data()

# using data after 2009
df_recent=main_raw[main_raw['year'] > 2010]

# using data that has more than 5 reviews
using_movies=df_recent.value_counts('title')[df_recent.value_counts('title') > 5].index

df_main=df_recent[df_recent['title'].isin(using_movies)].reset_index(drop=True)

In [23]:
print('total raitings : ',len(df_main))
print('total movies : ',len(df_main['title'].unique()))
print('total users : ',len(df_main['userId'].unique()))

total raitings :  28006
total movies :  1555
total users :  232


In [24]:
df_main.drop_duplicates('title').head()

,userId,movieId,rating,timestamp,title,genres,time_line,year,month
0,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2017-11-13_21:11,2017,11
63,18,6,4.0,1460138360,Heat (1995),Action|Crime|Thriller,2016-04-09_02:04,2016,4
80,15,47,3.5,1510571970,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,2017-11-13_20:11,2017,11
142,16,50,4.0,1377476781,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,2013-08-26_09:08,2013,8
214,18,70,3.5,1455735732,From Dusk Till Dawn (1996),Action|Comedy|Horror|Thriller,2016-02-18_04:02,2016,2


In [25]:
recommand_general(df_main,10)

,title,est
0,"Shawshank Redemption, The (1994)",4.335814
1,One Flew Over the Cuckoo's Nest (1975),4.293710
2,"Shining, The (1980)",4.291110
3,Life Is Beautiful (La Vita è bella) (1997),4.266347
4,Pulp Fiction (1994),4.256498
5,Goodfellas (1990),4.247644
6,Monty Python's Life of Brian (1979),4.246504
7,Fight Club (1999),4.239982
8,"Road Warrior, The (Mad Max 2) (1981)",4.236315
9,"Godfather, The (1972)",4.223760


In [26]:
recommand_with_genres(df_main,10,['Animation','Fantasy','Children'])

,title,est,genres
0,Toy Story 3 (2010),4.068198,"[Adventure, Animation, Children, Comedy, Fanta..."
1,My Neighbor Totoro (Tonari no Totoro) (1988),4.008832,"[Animation, Children, Drama, Fantasy]"
2,Laputa: Castle in the Sky (Tenkû no shiro Rapy...,3.978188,"[Action, Adventure, Animation, Children, Fanta..."
3,Toy Story 2 (1999),3.974984,"[Adventure, Animation, Children, Comedy, Fantasy]"
4,"Monsters, Inc. (2001)",3.965341,"[Adventure, Animation, Children, Comedy, Fantasy]"
5,Toy Story (1995),3.951217,"[Adventure, Animation, Children, Comedy, Fantasy]"
6,Beauty and the Beast (1991),3.944668,"[Animation, Children, Fantasy, Musical, Romanc..."
7,Fantasia (1940),3.943668,"[Animation, Children, Fantasy, Musical]"
8,How to Train Your Dragon (2010),3.941652,"[Adventure, Animation, Children, Fantasy, IMAX]"
9,Tangled (2010),3.929927,"[Animation, Children, Comedy, Fantasy, Musical..."


In [29]:
my_seen_movies=recommand_with_genres(df_main,10,['Animation','Fantasy','Children'])['title'].values

In [30]:
my_seen_movies

array(['Toy Story 3 (2010)',
       'My Neighbor Totoro (Tonari no Totoro) (1988)',
       'Laputa: Castle in the Sky (Tenkû no shiro Rapyuta) (1986)',
       'Monsters, Inc. (2001)', 'Toy Story (1995)', 'Toy Story 2 (1999)',
       'How to Train Your Dragon (2010)', 'Fantasia (1940)',
       'Tangled (2010)', 'Beauty and the Beast (1991)'], dtype=object)

In [31]:
recommand_with_movies(df_main,10,my_seen_movies)

,title,est
0,"Princess Bride, The (1987)",4.165540
1,Forrest Gump (1994),4.117494
2,"Lion King, The (1994)",4.072650
3,Toy Story 3 (2010),4.070223
4,WALL·E (2008),4.055467
5,Up (2009),4.051404
6,"Lord of the Rings: The Two Towers, The (2002)",3.985292
7,Ratatouille (2007),3.974749
8,Aladdin (1992),3.941152
9,Toy Story (1995),3.939535


In [32]:
df_main[df_main['title'].isin(my_seen_movies)].value_counts('userId')

userId
509    10
525     9
177     9
596     8
380     8
       ..
487     1
189     1
154     1
319     1
10      1
Length: 133, dtype: int64

In [33]:
check_users=df_main[df_main['title'].isin(my_seen_movies)]['userId'].unique()

In [34]:
(df_main[df_main['title'].isin(my_seen_movies)].value_counts('userId')/df_main[df_main['userId'].isin(check_users)].value_counts('userId')).nlargest(5)

userId
87     0.200000
606    0.166667
401    0.112903
471    0.111111
251    0.090909
dtype: float64

In [35]:
recommanded_movies=recommand_user_id_ensemble(df_main,87,200,weights=(0,0,0.5))

In [37]:
recommanded_movies.loc[recommanded_movies['genre_simillarity_point'].dropna().index,:].head(10)

,title,est,point,svd_est_point,movie_simillarity_point,genre_simillarity_point
0,Pulp Fiction (1994),4.252486,541.0,184.0,156.0,201.0
1,Fargo (1996),4.180994,490.5,182.0,109.0,199.5
2,Forrest Gump (1994),4.136989,428.5,100.0,132.0,196.5
3,Life Is Beautiful (La Vita è bella) (1997),4.286273,392.5,190.0,NaN,202.5
4,Trainspotting (1996),4.230609,390.0,192.0,NaN,198.0
6,In Bruges (2008),4.092245,354.5,161.0,NaN,193.5
11,Lost in Translation (2003),4.065702,320.0,137.0,NaN,183.0
12,Mary and Max (2009),4.115143,307.0,112.0,NaN,195.0
13,Fear and Loathing in Las Vegas (1998),4.060368,285.0,93.0,NaN,192.0
14,Little Miss Sunshine (2006),3.946313,276.5,101.0,NaN,175.5


In [38]:
recommanded_movies.loc[recommanded_movies['movie_simillarity_point'].dropna().index,:].head(10)

,title,est,point,svd_est_point,movie_simillarity_point,genre_simillarity_point
0,Pulp Fiction (1994),4.252486,541.0,184.0,156.0,201.0
1,Fargo (1996),4.180994,490.5,182.0,109.0,199.5
2,Forrest Gump (1994),4.136989,428.5,100.0,132.0,196.5
5,Fight Club (1999),4.232279,377.0,193.0,184.0,NaN
7,"Big Lebowski, The (1998)",3.978704,353.0,195.0,158.0,NaN
8,Goodfellas (1990),4.257937,353.0,199.0,154.0,NaN
9,"Matrix, The (1999)",4.107130,336.0,170.0,166.0,NaN
10,Reservoir Dogs (1992),4.181208,326.0,180.0,146.0,NaN
16,Seven (a.k.a. Se7en) (1995),4.166299,269.0,186.0,83.0,NaN
19,Predator (1987),4.166437,255.0,188.0,67.0,NaN


In [39]:
recommanded_movies.loc[recommanded_movies['svd_est_point'].dropna().index,:].head(10)

,title,est,point,svd_est_point,movie_simillarity_point,genre_simillarity_point
0,Pulp Fiction (1994),4.252486,541.0,184.0,156.0,201.0
1,Fargo (1996),4.180994,490.5,182.0,109.0,199.5
2,Forrest Gump (1994),4.136989,428.5,100.0,132.0,196.5
3,Life Is Beautiful (La Vita è bella) (1997),4.286273,392.5,190.0,NaN,202.5
4,Trainspotting (1996),4.230609,390.0,192.0,NaN,198.0
5,Fight Club (1999),4.232279,377.0,193.0,184.0,NaN
6,In Bruges (2008),4.092245,354.5,161.0,NaN,193.5
7,"Big Lebowski, The (1998)",3.978704,353.0,195.0,158.0,NaN
8,Goodfellas (1990),4.257937,353.0,199.0,154.0,NaN
9,"Matrix, The (1999)",4.107130,336.0,170.0,166.0,NaN


In [40]:
recommanded_movies[recommanded_movies['title'].apply(lambda x: True if int(x[-5:-1]) > 2010 else False)].head(10)

,title,est,point,svd_est_point,movie_simillarity_point,genre_simillarity_point
21,"Crazy, Stupid, Love. (2011)",4.026352,244.5,57.0,NaN,187.5
24,Inherent Vice (2014),3.665490,218.0,NaN,131.0,87.0
25,The Raid: Redemption (2011),4.005947,215.0,25.0,190.0,NaN
28,"Wolf of Wall Street, The (2013)",3.986600,212.5,31.0,NaN,181.5
40,Spotlight (2015),4.184600,194.0,194.0,NaN,NaN
41,"Fast Five (Fast and the Furious 5, The) (2011)",3.288562,193.0,NaN,193.0,NaN
47,Intouchables (2011),4.085803,190.5,NaN,NaN,190.5
52,"Avengers, The (2012)",3.855275,188.0,33.0,155.0,NaN
57,Neighbors (2014),3.629254,186.0,NaN,186.0,NaN
60,The Artist (2011),4.039544,184.5,NaN,NaN,184.5


In [41]:
title_genre_dict=df_main.drop_duplicates('title').loc[:,['title','genres']].set_index('title')['genres'].to_dict()

In [42]:
def genre_filter(df,genre):
    result=df[df['title'].map(title_genre_dict).apply(lambda x: True if genre in x.split('|') else False)]    
    return result

In [43]:
genre_filter(recommanded_movies,'Animation').head(10)

,title,est,point,svd_est_point,movie_simillarity_point,genre_simillarity_point
12,Mary and Max (2009),4.115143,307.0,112.0,NaN,195.0
53,"Little Mermaid, The (1989)",3.809517,188.0,NaN,188.0,NaN
84,"Girl Who Leapt Through Time, The (Toki o kaker...",3.951070,172.5,NaN,NaN,172.5
117,Ghost in the Shell (Kôkaku kidôtai) (1995),4.052781,159.0,159.0,NaN,NaN
119,Zootopia (2016),3.812964,158.0,158.0,NaN,NaN
122,Enchanted (2007),3.805669,157.0,157.0,NaN,NaN
131,Inside Out (2015),3.851098,153.0,NaN,NaN,153.0
140,Team America: World Police (2004),3.599945,149.0,NaN,149.0,NaN
142,Pinocchio (1940),3.716168,148.0,NaN,148.0,NaN
145,Snow White and the Seven Dwarfs (1937),3.648628,147.0,NaN,147.0,NaN
